# Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ESA-Datalabs/XAMI-model/blob/main/dataset_and_model.ipynb)

In [17]:
import sys
import os

if 'google.colab' in sys.modules:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

In [ ]:
# skip this step if the package already exists
!git clone https://github.com/ESA-Datalabs/XAMI-model.git
os.chdir("XAMI-model")
!conda env update -n base -f environment.yaml
!pip install -e .

fatal: destination path 'XAMI-model' already exists and is not an empty directory.
WARNING conda.models.version:get_matcher(537): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.7.1.*, but conda is ignoring the .* and treating it as 1.7.1
WARNING conda.models.version:get_matcher(537): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.9.0.*, but conda is ignoring the .* and treating it as 1.9.0
WARNING conda.models.version:get_matcher(537): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.6.0.*, but conda is ignoring the .* and treating it as 1.6.0
done
Solving environment: / 

## Download dataset

In [ ]:
!git clone https://github.com/ESA-Datalabs/XAMI-dataset.git
!pip install -e XAMI-dataset/

In [ ]:
# in colab, there is an issue with recognizing these packages 
if 'google.colab' in sys.modules: 
  sys.path.append(os.getcwd())
  sys.path.append(os.path.join(os.getcwd(), 'XAMI-dataset'))

In [ ]:
from xami_dataset.loader import XAMIDataset

# Download the dataset
xami_dataset = XAMIDataset(
    repo_id="iulia-elisa/XAMI-dataset",
    dataset_name="xami_dataset", # this is the COCO format of the dataset (can use 'xami_dataset_yolov8' for the yolov8 format)
    dest_dir='./data')

## Download model weights

In [ ]:
from huggingface_hub import hf_hub_download

dest_dir = './xami_model/train/weights/'

# the RT-DETR backbone performs better than YOLO (except on 'Other' class)
# YOLOv8n is faster and has a good speed-accuracy trade-off, with usually -10ms on inference compared to RT-DETR

det_type = 'yolov8' # 'yolov8' or 'rtdetr'

checkpoints = [
        f'{det_type}_sam_weights/{det_type}_detect_300e_best.pt', 
        f'{det_type}_sam_weights/{det_type}_detect_300e_last.pt', # this can help sometimes
        f'{det_type}_sam_weights/{det_type}_sam.pth']

# download checkpoints from HuggingFace
for checkpoint in checkpoints:
    hf_hub_download(
    			repo_id="iulia-elisa/XAMI-model",
    			repo_type='model',
    			filename=checkpoint,
    			local_dir=dest_dir
    		)
print(f'Checkpoints saved in {dest_dir}')

# Inference

In [ ]:
from xami_model.inference.xami_inference import InferXami

det_type = 'yolov8' # 'rtdetr' or 'yolov8' variants
detr_checkpoint = f'./xami_model/train/weights/{det_type}_sam_weights/{det_type}_detect_300e_best.pt'
sam_checkpoint = f'./xami_model/train/weights/{det_type}_sam_weights/{det_type}_sam.pth'

#❗️the SAM checkpoint and model_type (vit_h, vit_t, etc.) must be compatible
#❗️the detr_type and its checkpoints must be compatible
detr_sam_pipeline = InferXami(
    device='cuda:0',
    detr_checkpoint=detr_checkpoint,
    sam_checkpoint=sam_checkpoint,
    model_type='vit_t',
    use_detr_masks=True,
    detr_type=det_type) 

In [ ]:
# masks = detr_sam_pipeline.run_predict('./example_images/S0893811101_M.png', show_masks=True)
masks = detr_sam_pipeline.run_predict('./example_images/S0743200101_V.jpg', show_masks=True)

In [ ]:
masks = detr_sam_pipeline.run_predict('./example_images/euclid_artefacts.png', show_masks=True, det_conf=0.2)